## Getting the GIS data


http://www.census.gov/geo/maps-data/data/tiger-data.html

# Setting up The database

Setting up the database for use with PostGIS

### Install PostGIS
```
sudo apt-get update
sudo apt-get install -y postgis postgresql-9.5-postgis-2.2
``` 
If you're on a mac,

ensure those are the latest versions

Install the postgis extensions in PostGres:
```
psql
```

```
CREATE EXTENSION postgis; CREATE EXTENSION postgis_topology;
```

Other extensions for additional geographic functionality available.

Databases should now support geographic datatypes and functions.
Sources: 
http://www.saintsjd.com/2014/08/13/howto-install-postgis-on-ubuntu-trusty.html
http://postgis.net/install/
http://www.bostongis.com/PrinterFriendly.aspx?content_name=postgis_tut01


### Create and Load Geodatabase

http://www.bostongis.com/PrinterFriendly.aspx?content_name=postgis_tut01

Download your shapefiles.
You may need to prep them before loading some.

Figure out the SRID (spatial reference id) of the shapefiles
    for GerryPy data it is going probably be 4269
    This website is useful to find your srid: http://spatialreference.org/ref/epsg/4269/


Convert the Shapefiles to .sql:
        Some tutorials talk about the commands being in a hard to get to directory.  I had access to them right out of the box, and will proceed as though you have them too.
        Run the following command from the directory that holds the shapefule (made up of .shp, .shx, .prj, .dbf, .xml, all with the same name).
        
```
shp2pgsql -s <srid> <shapefile_name> [optional table name] > <new_name for .sql file>.sql
```

You should now have a .sql file in that same directory.  It contains the table and the geographic shapes.  This is what we'll add to the database.  


Load files to database:
```
psql -d <your_username> -h localhost -U <your_username> -f <your_new_.sql_file>
```

I sometimes have trouble with this last step.  If you're getting errors about the table not existing.  Try adding it to your username database.  then use sql to copy it to a new database.

Check your DB!  You should be ready to go.




## Finding the neighboring shapes with PostGIS

http://stackoverflow.com/questions/26866416/finding-neighbouring-polygons-postgis-query

The edges table is built by finding every set of bordering tracts.  The pos
```
Update gemstat_simple5 gem set spread=1, time=2
  FROM (
     SELECT (b."GEM_NR")
       FROM gemstat_simple5 as a,
            gemstat_simple5 as b
       WHERE ST_Touches(a.the_geom, b.the_geom)
       AND a."GEM_NR" != b."GEM_NR"
       AND a.spread = 1
     ) as subquery
 WHERE gem."GEM_NR" = subquery."GEM_NR"
```

## District View  SQL

In your psql database run the following SQL to build a view that generates summary stats and shapes fomr your tract table "colorado_tracts".

```
CREATE VIEW vwDistrict

AS

SELECT t.districtid,
       sum(t.tract_pop) as population,
       sum(t.shape_area) as area,
       cast(ST_MULTI(ST_UNION(t.geom)) as geometry(MultiPolygon,4269)) as geom
FROM colorado_tracts AS t
GROUP BY t.districtid;
```




## Getting PyQGIS working on your computer

We have to use python2 to work with QGIS (boo) 
We dont yet know how to do tht accross the board, but we'll figure that out.

After installing QGIS, open up its desktop software.  From the top toolbar: click Plugins > Python Console

In the console, type: 
```
import qgis
print(qgis.__file__)
```
That should give you the location of the qgis scripts.
it should look something like :
"/usr/lib/python2.7/dist-packages/qgis/__init__.pyc"

go to your venv activate script and set a new environment variable for that path up until the qgis module.
```
export PYTHONPATH=/usr/lib/python2.7/dist-packages/
```

Now, in your module, you should be able to
```
import qgis
```






## Setting up PyQGIS

PyQGIS needs to be able to access its modules, which are buried somewhere on your computer.  You probably wont be able to access them right away.  

If you go to http://docs.qgis.org/testing/en/docs/pyqgis_developer_cookbook/intro.html#using-pyqgis-in-custom-applications

There's a solution, but it didnt work for me.  My files are in a different directory.

Open up your QGIS Desktop app.  Click on Plugins > Python Console.  In console, type:
```
import qgis
print(qgis.__file__)
```
The retuen value should be something like.  /usr/lib/python2.7/dist-packages/qgis/__init__.pyc

Copy the path up to 'qgis' and go to your bin/activate file in your venv.

Set a new environment variable for your PYTHONPATH that leads to that location, like so:
export PYTHONPATH=/usr/lib/python2.7/dist-packages/

Restart your venv and pyqgis modules should work.  should...

##Working with Vectors

how to select by an attribute expression
http://gis.stackexchange.com/questions/131158/how-to-select-features-using-an-expression-with-pyqgis
